[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/transduction_dices/observation_prediction_dices.ipynb)

# Transduction from observation to prediction for dices

In [7]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction_dices.sh http://josd.github.io/eye/transduction/transduction_dices/observation_prediction_dices.sh
! curl -o observation_prediction_dices.py http://josd.github.io/eye/transduction/transduction_dices/observation_prediction_dices.py
! curl -o __init__.py http://josd.github.io/eye/transduction/transduction_dices/__init__.py
! curl -o sample_dices.observation http://josd.github.io/eye/transduction/transduction_dices/sample_dices.observation
! chmod +x observation_prediction_dices.sh
% rm -fr ~/t2t_data/observation_prediction_dices
% rm -fr ~/t2t_train/observation_prediction_dices/transformer-transformer_small/

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.2)
Requirement not upgraded as not directly required: google-api-python-client in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.6.7)
Requirement not upgraded as not directly required: gevent in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.3.0)
Requirement not upgraded as not directly required: flask in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.0.2)
Requirement not upgraded as not directly required: future in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.16.0)
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (2.7.1)
Requirement not upgraded as not directly required: gunicorn in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (19.8.1)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from tensor

In [8]:
# See the observation_prediction_dices problem

! pygmentize -g observation_prediction_dices.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionDices(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for dices."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      outcome = random.randint(1, 6)
      yield {
    

In [9]:
# See the observation_prediction_dices script

! pygmentize -g observation_prediction_dices.sh

#!/bin/bash
PROBLEM=observation_prediction_dices
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=1000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_dices.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_dices.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM 

In [10]:
# Run the observation_prediction_dices script

! ./observation_prediction_dices.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_dices
INFO:tensorflow:Generating data for observation_prediction_dices.
INFO:tensorflow:Generating vocab file: /content/t2t_data/observation_prediction_dices/vocab.observation_prediction_dices.16384.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 259
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 148
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 148
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 148
INFO:tensorflow:Trying min_count 250
INFO:tensorflow:Iteration 0

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa3cb3e8ac8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': 1234, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.75
}
allow_soft_placement: true
graph_options {
  optimizer_options {
  }
}
, '_save_checkpoints_steps': 100, '_kee

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_9138_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_9138_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_9138_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-20:49:40
INFO:tensorflow:Graph was finalized.
2018-05-13 20:49:40.367245: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 20:49:40.367350: I tensorflow/core/c

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_9138_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_9138_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_9138_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-20:50:34
INFO:tensorflow:Graph was finalized.
2018-05-13 20:50:34.794752: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 20:50:34.794860: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:50:34.794895: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 20:50:34.794925: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 20:50:34.795143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/tas

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-20:51:28
INFO:tensorflow:Graph was finalized.
2018-05-13 20:51:28.580156: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 20:51:28.580263: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:51:28.580295: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 20:51:28.580320: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 20:51:28.580520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /content/t2t_train/observation_prediction_dices/transformer-transformer_small/model.ckpt-300
INFO:tens

...

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-20:55:05
INFO:tensorflow:Graph was finalized.
2018-05-13 20:55:05.723446: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 20:55:05.723551: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:55:05.723582: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 20:55:05.723609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 20:55:05.723831: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /content/t2t_train/observation_prediction_dices/transformer-transformer_small/model.ckpt-700
INFO:tens

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-13-20:55:59
INFO:tensorflow:Graph was finalized.
2018-05-13 20:55:59.911019: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 20:55:59.911131: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:55:59.911163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 20:55:59.911190: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 20:55:59.911399: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /content/t2t_train/observation_prediction_dices/transformer-transformer_small/model.ckpt-800
INFO:tens

INFO:tensorflow:Starting evaluation at 2018-05-13-20:56:54
INFO:tensorflow:Graph was finalized.
2018-05-13 20:56:54.513656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 20:56:54.513779: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:56:54.513812: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 20:56:54.513841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 20:56:54.514131: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /content/t2t_train/observation_prediction_dices/transformer-transformer_small/model.ckpt-900
INFO:tensorflow:Running local_init_op.
INFO:tens

INFO:tensorflow:Graph was finalized.
2018-05-13 20:57:48.827250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-05-13 20:57:48.827351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-13 20:57:48.827382: I tensorflow/core/common_runtime/gpu/gpu_device.cc:917]      0 
2018-05-13 20:57:48.827408: I tensorflow/core/common_runtime/gpu/gpu_device.cc:930] 0:   N 
2018-05-13 20:57:48.827657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 8579 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /content/t2t_train/observation_prediction_dices/transformer-transformer_small/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluat

INFO:tensorflow:BEAM 0:
INFO:tensorflow:Inference results INPUT: A_THROW t100005
INFO:tensorflow:Inference results OUTPUT: outcome 3
INFO:tensorflow:BEAM 1:
INFO:tensorflow:Inference results INPUT: A_THROW t100005
INFO:tensorflow:Inference results OUTPUT: outcome 6
INFO:tensorflow:BEAM 2:
INFO:tensorflow:Inference results INPUT: A_THROW t100005
INFO:tensorflow:Inference results OUTPUT: outcome 4
INFO:tensorflow:BEAM 0:
INFO:tensorflow:Inference results INPUT: A_THROW t100004
INFO:tensorflow:Inference results OUTPUT: outcome 3
INFO:tensorflow:BEAM 1:
INFO:tensorflow:Inference results INPUT: A_THROW t100004
INFO:tensorflow:Inference results OUTPUT: outcome 6
INFO:tensorflow:BEAM 2:
INFO:tensorflow:Inference results INPUT: A_THROW t100004
INFO:tensorflow:Inference results OUTPUT: outcome 4
INFO:tensorflow:BEAM 0:
INFO:tensorflow:Inference results INPUT: A_THROW t100003
INFO:tensorflow:Inference results OUTPUT: outcome 3
INFO:tensorflow:BEAM 1:
INFO:tensorflow:Inference results INPUT: A_TH

In [11]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_dices.observation
print("->-")
! pygmentize -g sample_dices.prediction

A_THROW t100001
A_THROW t100002
A_THROW t100003
A_THROW t100004
A_THROW t100005
A_THROW t100006
->-
outcome 3	-1.63	outcome 4	-1.64	outcome 6	-1.65
outcome 3	-1.62	outcome 6	-1.64	outcome 4	-1.67
outcome 3	-1.62	outcome 6	-1.65	outcome 4	-1.67
outcome 3	-1.62	outcome 6	-1.65	outcome 4	-1.67
outcome 3	-1.60	outcome 6	-1.64	outcome 4	-1.68
outcome 3	-1.61	outcome 6	-1.63	outcome 4	-1.67
